<a href="https://colab.research.google.com/github/sylpurnama/Data-Services/blob/master/NASA_FIRMS_API_Fire_Hotspots_in_Indonesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster

In [ ]:
MAP_KEY = '430c52106a780ccb3cd78318ad2dd3b9'

In [ ]:
# this url will return information about all supported sensors and their corresponding datasets
# instead of 'all' you can specify individual sensor, ex:LANDSAT_NRT
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df = pd.read_csv(da_url)
display(df)

,data_id,min_date,max_date
0,MODIS_NRT,2024-12-01,2025-03-06
1,MODIS_SP,2000-11-01,2024-11-30
2,VIIRS_NOAA20_NRT,2024-12-01,2025-03-06
3,VIIRS_NOAA20_SP,2018-04-01,2024-11-30
4,VIIRS_NOAA21_NRT,2024-01-17,2025-03-06
5,VIIRS_SNPP_NRT,2025-01-01,2025-03-06
6,VIIRS_SNPP_SP,2012-01-20,2024-12-31
7,LANDSAT_NRT,2022-06-20,2025-03-06
8,GOES_NRT,2022-08-09,2025-03-07
9,BA_MODIS,2000-11-01,2024-11-01


In [ ]:
# Function to fetch MODIS hotspots by country
def get_hotspots_by_country(country_code, day_range):
    url = f'https://firms.modaps.eosdis.nasa.gov/api/country/csv/{MAP_KEY}/MODIS_NRT/{country_code}/{day_range}'
    try:
        df = pd.read_csv(url)
        print(f"Retrieved {len(df)} records for country {country_code}")
        return df
    except:
        print("Error retrieving data for the specified country.")
        return None

In [ ]:
#Indonesia FIRES
indonesia_url = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + '/MODIS_NRT/IDN/6'
df_indonesia = pd.read_csv(indonesia_url)
df_indonesia

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,IDN,-7.05063,107.76369,310.61,3.14,1.68,2025-03-02,247,Terra,MODIS,58,6.1NRT,290.03,30.02,D
1,IDN,-7.04722,107.73623,311.53,3.13,1.68,2025-03-02,247,Terra,MODIS,65,6.1NRT,290.36,32.16,D
2,IDN,-7.04719,107.74290,313.20,3.13,1.68,2025-03-02,247,Terra,MODIS,56,6.1NRT,290.58,35.80,D
3,IDN,-6.97089,107.61507,315.21,3.05,1.66,2025-03-02,247,Terra,MODIS,49,6.1NRT,291.56,26.10,D
4,IDN,-6.14492,106.76616,315.45,2.52,1.53,2025-03-02,247,Terra,MODIS,32,6.1NRT,290.93,25.99,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,IDN,-2.01174,121.41445,316.82,1.00,1.00,2025-03-06,629,Aqua,MODIS,51,6.1NRT,293.60,9.56,D
85,IDN,-0.52963,115.33630,309.89,1.91,1.35,2025-03-06,629,Aqua,MODIS,45,6.1NRT,288.48,13.87,D
86,IDN,-0.52719,115.35287,309.71,1.91,1.35,2025-03-06,629,Aqua,MODIS,48,6.1NRT,287.66,13.56,D
87,IDN,-0.38572,116.71312,306.81,1.49,1.21,2025-03-06,629,Aqua,MODIS,55,6.1NRT,276.48,7.99,D


In [ ]:
df = df_indonesia

In [ ]:
# Initialize Folium Map
m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=5)
marker_cluster = MarkerCluster().add_to(m)

# Add fire markers
for _, row in df.iterrows():
    popup_text = (f"<b>Brightness Temp:</b> {row['brightness']}<br>"
                  f"<b>Confidence:</b> {row['confidence']}<br>"
                  f"<b>FRP:</b> {row['frp']}<br>"
                  f"<b>Date:</b> {row['acq_date']}<br>"
                  f"<b>Satellite:</b> {row['satellite']}")
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="red", icon="fire")
    ).add_to(marker_cluster)

# Display the map
m